<br>
<img align="center" width="500" src="rflogo.png">
<br>

# Optimizing the Tenant Mix
<br>
<img align="left" width="80" height="200" src="https://img.shields.io/badge/python-v3.6-blue.svg">
<br>

<br>

### Notebook by [Marco Tavora](https://marcotavora.me/)

## Table of contents
 
1. [Import Modules](#Import-Modules)
2. [Problem](#Problem)
3. [Steps](#Steps)
4. [Optimal Tenant Mix](#Optimal-Tenant-Mix)
5. [Mathematical Problem](#Mathematical-Problem)
3. [Cleaning up](#Cleaning-up)
4. [Dictionary from codes to variables](#Dictionary-from-codes-to-variables)
5. [Defining dimensions](#Defining-dimensions)
6. [Generating artificial parameters to build code](#Generating-artificial-parameters-to-build-code)
7. [Code](#Code)

### Import Modules  
[[go back to the top]](#Table-of-contents)

In [18]:
%run modules_tenant_optimizer.ipynb

### Problem 
[[go back to the top]](#Table-of-contents)

We will mostly follow [this article](https://dl.acm.org/citation.cfm?id=2752405).

The development of a new shopping mall follows these phases:
- Large department stores are accomodated first after some negotiation
- After the department stores are situated and a general mall floorplan is known, decisions regarding how to lease the many remaining smaller store spaces are taken. In this phase, the types, sizes, and locations of these smaller tenants are crucial in determining the mall's financial success. 

The tenant mix problem is formulated as a nonlinear integer program and solved it using a linear approximation. 

### Steps
[[go back to the top]](#Table-of-contents)

- Evaluate feasibility considering e.g. marketing demographics, building costs, and expected revenues. 
- A tentative floor plan including size, shape, and spaces for large department stores is outlined. Leasing agreements are reached with the two or three major department stores (anchor stores). The latter determine the character of the mall and provide the visibility needed. They usually negotiate highly favorable occupancy agreements (low rent and/or other concessions). The mall's profits are made primarily from the rent paid by the non-anchor tenants, i.e. the smaller stores that lease space along the aisles of the mall. 

### Optimal Tenant Mix
[[go back to the top]](#Table-of-contents)

The optimal tenant mix along with stores' sizes and locations, would, by definition, make the mall attractive to consumers and profitable for the retailers and for the developer. 

### Cleaning up
[[go back to the top]](#Table-of-contents)

In [3]:
file = 'shopping_ficticio_v6.xlsx'
xl = pd.ExcelFile(file)
print('sheet names:',xl.sheet_names)
(shopping, lojas, varej) = (xl.parse(xl.sheet_names[0]), 
                           xl.parse(xl.sheet_names[1], skiprows=1), 
                           xl.parse(xl.sheet_names[2]))

sheet names: ['Shopping', 'Lojas', 'Varejistas']


In [4]:
lojas.head(2)

,Loja,Piso,Corredor (J = 3 (location classes)),Tipo de Loja,Loja Esquerda,Loja Direita,Loja em Frente,Área (m2),K = 5 (store size classes),Vitrine (m),Esquina (Sim/Não),Ocupante,Segmento (I = 21 (store types)),Tipo de Segmento,Faturamento Mensal,Ticket Médio,Conv. Corredor/Loja,Conv. Loja/Compras,Fluxo Diário no Corredor,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,1,1,Corredor 01,Satélite,Entrada 01 Esquerda,3,106,294.0,4.0,12.40,Não,Ótica São Pedro,Ótica,Impulso,65610.0,45.0,0.05,0.15,6480.0,NaN,NaN,NaN,NaN,NaN,Store size class,Store size class,Store size class,Store size class,Store size class,Location class,Location class,Location class
1,2,1,Corredor 03,Alimentação,84,85,71,44.0,1.0,5.68,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Store Types,Num Lojas,Total ABL (m2),1,2,3,4,5,Corredor 01,Corredor 02,Corredor 03


In [5]:
nan_lst = []
n = 5
for s in lojas.columns:
    lst = [s, round(lojas[s].isna().sum()/lojas.shape[0],2)]
    if lst[1] == 1.0:
        nan_lst.append(lst)
cols_to_drop = [el[0] for el in nan_lst] + ['Unnamed: 20']
lojas = lojas.drop(cols_to_drop, axis=1)
lojas.head(2)

,Loja,Piso,Corredor (J = 3 (location classes)),Tipo de Loja,Loja Esquerda,Loja Direita,Loja em Frente,Área (m2),K = 5 (store size classes),Vitrine (m),Esquina (Sim/Não),Ocupante,Segmento (I = 21 (store types)),Tipo de Segmento,Faturamento Mensal,Ticket Médio,Conv. Corredor/Loja,Conv. Loja/Compras,Fluxo Diário no Corredor,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,1,1,Corredor 01,Satélite,Entrada 01 Esquerda,3,106,294.0,4.0,12.40,Não,Ótica São Pedro,Ótica,Impulso,65610.0,45.0,0.05,0.15,6480.0,NaN,NaN,NaN,Store size class,Store size class,Store size class,Store size class,Store size class,Location class,Location class,Location class
1,2,1,Corredor 03,Alimentação,84,85,71,44.0,1.0,5.68,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Store Types,Num Lojas,Total ABL (m2),1,2,3,4,5,Corredor 01,Corredor 02,Corredor 03


#### Tabela 1

In [6]:
col_to_crop = lojas.columns.tolist().index('Unnamed: 21')
lojas_1 = lojas.iloc[:,:col_to_crop]
lojas_1.dropna(inplace=True)
lojas_1.columns = [un_st(x.lower().replace('(','').replace(')',''))
                    for x in lojas_1.columns]
lojas_1.head(2)

,loja,piso,corredor j = 3 location classes,tipo de loja,loja esquerda,loja direita,loja em frente,area m2,k = 5 store size classes,vitrine m,esquina sim/nao,ocupante,segmento i = 21 store types,tipo de segmento,faturamento mensal,ticket medio,conv. corredor/loja,conv. loja/compras,fluxo diario no corredor
0,1,1,Corredor 01,Satélite,Entrada 01 Esquerda,3,106,294.0,4.0,12.4,Não,Ótica São Pedro,Ótica,Impulso,65610.0,45.0,0.05,0.150,6480.0
2,3,1,Corredor 01,Satélite,1,11,105,40.0,1.0,6.0,Não,CVC,Serviço,Serviço,218700.0,750.0,0.02,0.075,6480.0


In [7]:
lojas_1.isnull().any().unique()

array([False])

#### Tabela 2

In [8]:
lojas_2 = lojas.iloc[:, col_to_crop:]
lojas_2.iloc[1,:] = ['Store Types', 'Num Lojas', 'Total ABL (m2)', 'Store size class_1', 'Store size class_2',
                   'Store size class_3', 'Store size class_4', 'Store size class_5', 
                   'Location class Corredor 01', 'Location class Corredor 02', 'Location class Corredor 03']
lojas_2 = lojas_2.iloc[1:,:]
lojas_2.columns = lojas_2.iloc[0,:]
lojas_2 = lojas_2.iloc[1:22, :]
lojas_1.dropna(inplace=True)
lojas_2.columns = [un_st(x.lower().replace(' ','_').replace('(','').replace(')',''))
                    for x in lojas_2.columns]
lojas_2['store_types'] = [un_st(x.lower().replace('.', ' ').replace('(','').replace(')','')) 
                          for x in list(lojas_2['store_types'])]

In [9]:
lojas_2.columns = ['store_types', 'num_lojas', 'total_abl_m2', 'size_1', 'size_2', 'size_3', 'size_4', 'size_5',
           'corredor_01', 'corredor_02', 'corredor_03']

In [10]:
lojas_2.head()

,store_types,num_lojas,total_abl_m2,size_1,size_2,size_3,size_4,size_5,corredor_01,corredor_02,corredor_03
2,acessorios femininos,4,273,1,3,0,0,0,3,1,0
3,alimentacao,16,1154,7,7,2,0,0,0,0,16
4,art esportivos,1,679,0,0,0,0,1,1,0,0
5,brinquedos,1,72,0,1,0,0,0,0,0,1
6,cafeteria,2,115,1,1,0,0,0,1,1,0


In [11]:
lojas_2.isnull().any().unique()

array([False])

In [12]:
types_original = list(lojas_2['store_types'])
lst = []
for x in list(range(1,len(types_original)+1)):
    lst.append('t'+ str(x))
types = lst
data = dict(zip(types_original, lst))
types_df = pd.DataFrame(list(data.items()), 
                        columns=['type_code', 'type'])

locations_original = ['corredor_01', 'corredor_02', 'corredor_03']
locations_df = pd.DataFrame(list(dict(zip(locations_original, ['c1', 'c2', 'c3'])).items()), 
                            columns=['location_code', 'location'])
sizes_original = ['size_1', 'size_2', 'size_3', 'size_4', 'size_5']
sizes_df = pd.DataFrame(list(dict(zip(sizes_original, ['s1', 's2', 's3', 's4', 's5'])).items()), 
                            columns=['size_code', 'size'])

### Dictionary from codes to variables
[[go back to the top]](#Table-of-contents)

In [13]:
print('type codes:')
types_df
print('location codes:')
locations_df
print('size codes:')
sizes_df

type codes:


,type_code,type
0,acessorios femininos,t1
1,alimentacao,t2
2,art esportivos,t3
3,brinquedos,t4
4,cafeteria,t5
5,calcados,t6
6,cama e banho,t7
7,colchoes,t8
8,departamento,t9
9,drogaria,t10


location codes:


,location_code,location
0,corredor_01,c1
1,corredor_02,c2
2,corredor_03,c3


size codes:


,size_code,size
0,size_1,s1
1,size_2,s2
2,size_3,s3
3,size_4,s4
4,size_5,s5


### Defining dimensions
[[go back to the top]](#Table-of-contents)

In [15]:
I, J, K = len(types_original), len(locations_original), len(sizes_original)

stores = [(i, j, k) for i in range(I) 
          for j in range(J) 
          for k in range(K)]

print('number of x:', len(stores))
for x in [[I, 'types'], [J, 'locations'],[K, 'sizes']]:
    print('number of {} classes is:'.format(x[1]),'is', x[0])

number of x: 315
number of types classes is: is 21
number of locations classes is: is 3
number of sizes classes is: is 5


### Generating artificial parameters to build code
[[go back to the top]](#Table-of-contents)

For simplicity we choose $I,J$ and $K$ equal to 2. But **the code is valid for any values of I, J, K**.

In [23]:
toy = input("Use toy model? ")

if toy == 'yes':
    I, J, K = 2, 2, 2
    stores = [(i, j, k) for i in range(I) 
              for j in range(J) 
              for k in range(K)]

    print('number of x:', len(stores))
    for x in [[I, 'types'], [J, 'locations'],[K, 'sizes']]:
        print('number of {} classes is:'.format(x[1]),'is', x[0])

Use toy model? yes
number of x: 8
number of types classes is: is 2
number of locations classes is: is 2
number of sizes classes is: is 2


The parameters to be chosen are:

\begin{eqnarray}
&&A_{ik} = \text{amount of area required for a store of type } i\,\, \text{and size} \,\,k.\nonumber\\
&&G_{j} = \text{total amount of square feet (gross leaseable area) available in location class} \,\,j\nonumber\\
&&f_i = \text{least amount of square feet available for type} \,\,i\nonumber\\
&&F_i = \text{largest amount of square feet available for type} \,\,i \nonumber\\
&&L_i = \text{amount of finishing allowance given to a tenant of type} \,\,i\,\, \text{per square foot leased}.\nonumber\\
&&B = \text{tenant allowance budget}\nonumber\\
&&M_i = \text{maximum number of tenants of type} \,\,i.\nonumber\\
&&m_i = \text{minimum number of tenants of type} \,\,i.\nonumber\\
&&N_s = \text{maximum number of small stores}\nonumber\\
&&S_{il} = \text{binary variable set to 1 if there are at least} \,\,l \,\,\text{stores of type} \,\,i\nonumber\\
&& {\text{PW}}_{ijkl} = \text{present worth of a store de scribed by} \,\,(i, j, k) \,\,\text{if it is one of} \,\,l \,\,\text{stores of type} \,\,i\nonumber\\
&& R_{il} = \text{average marginal revenue added by including the}\,\, l\text{-th} \,\,\text{store of type}\,\, i\nonumber\\
\end{eqnarray}

### 1) ${\text{PW}}_{ijkl}$

The components of the income stream from a given store are illustrated below. The only cost associated with each store is an interior finishing allowance, which consists of a single payment at time zero. From then on, the model assumes that the store makes a single payment of rent at the end of each year. The final year's rent is multiplied by a constant to obtain the store's contribution to the sale value.
The authors have data that allows them to estimate a store's sales over the study horizon given its type, size, location class, and the number of stores of its type in the mall. These revenue figures, together with rental rates depending on the store's characteristics, determine the rental income the owner will receive. 

This quantity is ${\text{PW}}_{ijkl}$ where the first three indices are identify the segment, location and size of the store. The $l$ index is the number of stores of type $i$. 

The index $l$ depends on $i$ and varies within the range $[m_i, M_i]$ where $m_i$ is the minimum number of tenants of type $i$ and $M_i$ is the maximum.

<br>
<img align="center" width="300" src="fw.png">
<br>

### Generating data by hand
[[go back to the top]](#Table-of-contents)

In [51]:
types_toy_model = types_df.iloc[:2,:]
locations_toy_model = locations_df.iloc[:2,:]
sizes_toy_model = sizes_df.iloc[:2,:]
types_toy_model['idx'] = list(types_toy_model.index)
locations_toy_model['idx'] = list(locations_toy_model.index)
sizes_toy_model['idx'] = list(sizes_toy_model.index)
types_toy_model
sizes_toy_model
locations_toy_model

,type_code,type,idx
0,acessorios femininos,t1,0
1,alimentacao,t2,1


,size_code,size,idx
0,size_1,s1,0
1,size_2,s2,1


,location_code,location,idx
0,corredor_01,c1,0
1,corredor_02,c2,1


In [108]:
FW = {(0, 0, 0): 10,
      (0, 0, 1): 11,
      (0, 1, 0): 12,
      (0, 1, 1): 9,
      (1, 0, 0): 13,
      (1, 0, 1): 17,
      (1, 1, 0): 10,
      (1, 1, 1): 8}

NS = 12
stores = [(i, j, k) for i in range(I) 
          for j in range(J) 
          for k in range(K)]
print('stores: ',stores,'\n')

random.seed(123)
types_and_sizes = [(i, k) for i in range(I) for k in range(K)]
A = random.sample(range(5, 10), I*K)
A_list = A.copy()
A = dict(zip(types_and_sizes, A))
for i in range(I):
    for k in range(K):
        print('Area required by store of type {} and size {}'.format(i, k),'is',A[i, k])
        
print('\n')

random.seed(234)
locations_classes = [j for j in range(J)]
G = random.sample(range(5, 40), J)
G = dict(zip(locations_classes, G))

for j in range(J):
    print('Area available in corredor {}'.format(j),'is', G[j])
    
print('')

type_classes = [i for i in range(I)]
F = [el+1 for el in A_list]
f = [el-1 for el in A_list]
F = dict(zip(type_classes, F))
f = dict(zip(type_classes, f))
for i in range(I):
    print('largest area available for type {}'.format(i),'is', F[i])
    print('smallest area available for type {}'.format(i),'is', f[i])
    
print('')

def parameter_values_ranges(lst, num_seed, min_range, max_range, d):
    random.seed(num_seed)
    aux = random.sample(range(min_range, max_range), len(lst))
    aux = [el-d for el in aux]
    return dict(zip(lst, aux))

M = parameter_values_ranges([i for i in range(I)], 24, 3, 5, 0)
m = parameter_values_ranges([i for i in range(I)], 23, 1, 3, 0)


R = {(0, 2): 10, (0, 3): 5, (0, 4): 2, (1, 1): 15, (1, 2): 10, (1, 3): 5}
S = {(0, 2): 1, (0, 3): 0, (0, 4): 0, (1, 1): 1, (1, 2): 0, (1, 3): 0}

for i in [el[0] for el in list(R.keys())]:
    for l in range(m[i], M[i]+1):
        print('Average marginal revenue added by including the {}-th store of type {}'.format(l, i),
                  'is', R[i, l])
    
print('')

lst = [3, 2]


for i in range(I):
    for el in lst:
        print('There are {} stores of type {}'. format(el, i))

print('')

for i in [el[0] for el in list(R.keys())]:
    for l in range(m[i], M[i]+1):
        print('Binary variable equal to 1 if there are at least {} stores of type {}'.format(l, i),
                  'is', S[i, l])
print('')
print('S is:', S)

stores:  [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)] 

Area required by store of type 0 and size 0 is 5
Area required by store of type 0 and size 1 is 7
Area required by store of type 1 and size 0 is 9
Area required by store of type 1 and size 1 is 6


Area available in corredor 0 is 26
Area available in corredor 1 is 21

largest area available for type 0 is 6
smallest area available for type 0 is 4
largest area available for type 1 is 8
smallest area available for type 1 is 6

Average marginal revenue added by including the 2-th store of type 0 is 10
Average marginal revenue added by including the 3-th store of type 0 is 5
Average marginal revenue added by including the 4-th store of type 0 is 2
Average marginal revenue added by including the 2-th store of type 0 is 10
Average marginal revenue added by including the 3-th store of type 0 is 5
Average marginal revenue added by including the 4-th store of type 0 is 2
Average marginal revenue a

The appropriate values of

    (FW, A, G, F, f, L, M, m, R, S, B, NS)
    
must be chosen. 

#### Equations from the paper

\[\begin{array}{l}
{\text{total income}} = \sum\limits_{i = 1}^I {\sum\limits_{l = {m_i}}^{{M_i}} {{R_{il}}{S_{il}}} }
\end{array}\]

In [109]:
total_income = 0
for i in range(I):
    total_income += sum([R[i, l] * S[i, l] for l in range(m[i], M[i]+1)])

\[\begin{array}{l}
{({\rm{dict\_S)}}_i}{\rm{ }} = \,\sum\limits_{l = {m_i}}^{{M_i}} {{S_{il}}} 
\end{array}\]

In [110]:
dict_S = {}
for i in range(I):
    dict_S[i] = sum([S[i, l] for l in range(m[i], M[i]+1)])

In [111]:
total_income
dict_S

25

{0: 1, 1: 1}

### Code
[[go back to the top]](#Table-of-contents)

In [142]:
def tenant_opt():

#     x = pulp.LpVariable.dicts('x', stores, lowBound = 0, cat = pulp.LpInteger)
    x = pulp.LpVariable.dicts('x', stores, lowBound = 0, upBound=I*J*K, cat = pulp.LpInteger)

    prob1 = pulp.LpProblem('TenantOptimization', LpMaximize)
    prob1 += sum([x[i, j, k] * FW[i, j, k] 
                  + total_income/(I*J*K) 
                  for i in range(I) 
                  for j in range(J) 
                  for k in range(K)])

    for j in range(J):
        prob1 += lpSum([A[i, k] * x[i, j, k] for i in range(I) for k in range(K)]) <= G[j]

    for i in range(I):
        prob1 += lpSum([A[i, k] * x[i, j, k] for j in range(J) for k in range(K)]) <= F[i]  
        prob1 += lpSum([A[i, k] * x[i, j, k] for j in range(J) for k in range(K)]) >= f[i] 
        prob1 += lpSum([x[i, j, k] for j in range(J) for k in range(K)]) <= M[i]  
        prob1 += lpSum([x[i, j, k] for j in range(J) for k in range(K)]) >= m[i] 
        prob1 += lpSum([x[i, j, k] for i in range(I) for j in range(J) for k in range(K)]) == dict_S[i]

    prob1 += lpSum([x[i, j, 1] for i in range(I) for j in range(J)]) <= NS 
    
    for i in range(I):
        for l in range(m[i], M[i]):
            prob1 += S[i, l] >= S[i, l+1]
            
    prob1 += lpSum([x[i, j, k] for i in range(I) for j in range(J) for k in range(K)]) == dict_S[i]
    
    return prob1

prob1 = tenant_opt()
prob1

TenantOptimization:
MAXIMIZE
10*x_(0,_0,_0) + 11*x_(0,_0,_1) + 12*x_(0,_1,_0) + 9*x_(0,_1,_1) + 13*x_(1,_0,_0) + 17*x_(1,_0,_1) + 10*x_(1,_1,_0) + 8*x_(1,_1,_1) + 25.0
SUBJECT TO
_C1: 5 x_(0,_0,_0) + 7 x_(0,_0,_1) + 9 x_(1,_0,_0) + 6 x_(1,_0,_1) <= 26

_C2: 5 x_(0,_1,_0) + 7 x_(0,_1,_1) + 9 x_(1,_1,_0) + 6 x_(1,_1,_1) <= 21

_C3: 5 x_(0,_0,_0) + 7 x_(0,_0,_1) + 5 x_(0,_1,_0) + 7 x_(0,_1,_1) <= 6

_C4: 5 x_(0,_0,_0) + 7 x_(0,_0,_1) + 5 x_(0,_1,_0) + 7 x_(0,_1,_1) >= 4

_C5: x_(0,_0,_0) + x_(0,_0,_1) + x_(0,_1,_0) + x_(0,_1,_1) <= 4

_C6: x_(0,_0,_0) + x_(0,_0,_1) + x_(0,_1,_0) + x_(0,_1,_1) >= 2

_C7: x_(0,_0,_0) + x_(0,_0,_1) + x_(0,_1,_0) + x_(0,_1,_1) + x_(1,_0,_0)
 + x_(1,_0,_1) + x_(1,_1,_0) + x_(1,_1,_1) = 1

_C8: 9 x_(1,_0,_0) + 6 x_(1,_0,_1) + 9 x_(1,_1,_0) + 6 x_(1,_1,_1) <= 8

_C9: 9 x_(1,_0,_0) + 6 x_(1,_0,_1) + 9 x_(1,_1,_0) + 6 x_(1,_1,_1) >= 6

_C10: x_(1,_0,_0) + x_(1,_0,_1) + x_(1,_1,_0) + x_(1,_1,_1) <= 3

_C11: x_(1,_0,_0) + x_(1,_0,_1) + x_(1,_1,_0) + x_(1,_1,_1) >= 1

In [140]:
prob1.writeLP("TenantOptimization.lp")

print(prob1.solve())

print("Status:", LpStatus[prob1.status])

lst_v = []
for v in prob1.variables():
    print(v.name, "=", v.varValue)
    lst_v.append(v.varValue)
    
print(FW)

v.varValue
lst_v

-1
Status: Infeasible
x_(0,_0,_0) = 0.0
x_(0,_0,_1) = 0.0
x_(0,_1,_0) = 2.0
x_(0,_1,_1) = 0.0
x_(1,_0,_0) = 4.0
x_(1,_0,_1) = -5.0
x_(1,_1,_0) = 0.0
x_(1,_1,_1) = 0.0
{(0, 0, 0): 10, (0, 0, 1): 11, (0, 1, 0): 12, (0, 1, 1): 9, (1, 0, 0): 13, (1, 0, 1): 17, (1, 1, 0): 10, (1, 1, 1): 8}


0.0

[0.0, 0.0, 2.0, 0.0, 4.0, -5.0, 0.0, 0.0]

In [141]:
array_v = np.array(lst_v)
array_FW = np.array(list(FW.values()))
array_FW
array_v
array_v.dot(array_FW)

array([10, 11, 12,  9, 13, 17, 10,  8])

array([ 0.,  0.,  2.,  0.,  4., -5.,  0.,  0.])

-9.0